The goal of this notebook is to explore the bug documented in https://github.com/desihub/desispec/pull/2057.

John Moustakas  
Siena College  
2023 June

In [1]:
import os
import numpy as np
import fitsio
from glob import glob
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table, vstack, join
from desitarget.targets import decode_targetid
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
default_vacdir = '/global/cfs/cdirs/desi/public/edr/vac/edr/lsdr9-photometry/fuji/v2.0/observed-targets'
bugfix_vacdir = '/global/cfs/cdirs/desi/public/edr/vac/edr/lsdr9-photometry/fuji/v2.1/observed-targets'

In [3]:
targetphot = Table(fitsio.read(default_vacdir+'/targetphot-sv1-fuji.fits'))

In [4]:
def read_tractorphot(zcat, racolumn='TARGET_RA', deccolumn='TARGET_DEC', verbose=False, vacdir=default_vacdir):
    from glob import glob
    from desimodel.footprint import radec2pix

    tractorphotfiles = glob(os.path.join(vacdir, 'tractorphot', 'tractorphot-nside4-hp???-fuji.fits'))
    
    hdr = fitsio.read_header(tractorphotfiles[0], 'TRACTORPHOT')
    tractorphot_nside = hdr['FILENSID']

    pixels = radec2pix(tractorphot_nside, zcat[racolumn], zcat[deccolumn])
    phot = []
    for pixel in set(pixels):
        J = pixel == pixels
        photfile = os.path.join(vacdir, 'tractorphot', 'tractorphot-nside4-hp{:03d}-fuji.fits'.format(pixel))
        if not os.path.isfile(photfile):
            #print(f'Missing file {photfile}!')
            continue
        targetids = fitsio.read(photfile, columns='TARGETID')
        K = np.where(np.isin(targetids, zcat['TARGETID'][J]))[0]
        
        if verbose:
            print('Reading photometry for {} objects from {}'.format(len(K), photfile))
            
        _phot = fitsio.read(photfile, rows=K)
        phot.append(Table(_phot))
        
    if len(phot) == 0:
        return Table()
        
    phot = vstack(phot)

    # Is there a better way to sort here??
    srt = np.hstack([np.where(tid == phot['TARGETID'])[0] for tid in zcat['TARGETID']]) 
    phot = phot[srt]
    
    return phot

In [5]:
#for tileid in [80611]:
for tileid in sorted(np.unique(targetphot['TILEID'])):
    #print(tileid)
    I = np.where(tileid == targetphot['TILEID'])[0]
    old = read_tractorphot(targetphot[I], racolumn='RA', deccolumn='DEC', vacdir=default_vacdir)
    new = read_tractorphot(targetphot[I], racolumn='RA', deccolumn='DEC', vacdir=bugfix_vacdir)
    
    # not all targets have tractor photometry (e.g., secondary targets)
    # and some tiles (e.g., 80715) have no photometry at all
    if len(old) == 0:
        continue

    keep = np.isin(targetphot[I]['TARGETID'], old['TARGETID'])
    #if np.sum(keep) != len(I):
    #    print('TileID {}: ignoring {}/{} targets with no Tractor photometry'.format(
    #        tileid, len(I)-np.sum(keep), len(I)))
    targetphot_ontile = targetphot[I][keep]
    targetphot_coord = SkyCoord(targetphot_ontile['RA']*u.degree, targetphot_ontile['DEC']*u.degree, frame='icrs')    
    
    old_coord = SkyCoord(old['RA']*u.degree, old['DEC']*u.degree, frame='icrs')
    new_coord = SkyCoord(new['RA']*u.degree, new['DEC']*u.degree, frame='icrs')    

    sep_old = old_coord.separation(targetphot_coord)
    sep_new = new_coord.separation(targetphot_coord)
    
    O = np.where(sep_old.arcsec >= 1.0)[0]
    N = np.where(sep_new.arcsec >= 1.0)[0]
    
    if len(O) > 0 or len(N) > 0:
        print('{} Nold={}/{} ({:.2f}%) Nnew={}/{} ({:.2f}%)'.format(# [{}/{} targets with no DR9 photometry]'.format(
            tileid, len(O), len(old), 100*len(O)/len(old), len(N), len(new), 100*len(N)/len(new)))#,
            #len(I)-np.sum(keep), len(I)))
    
    if len(O) > 0:
        out_old = join(targetphot_ontile[O]['TARGETID', 'RA', 'DEC', 'RELEASE', 'BRICKNAME', 'BRICKID', 'BRICK_OBJID'],
                       old[O]['TARGETID', 'RA', 'DEC', 'RELEASE', 'BRICKNAME', 'BRICKID', 'OBJID', 'LS_ID'],
                       keys='TARGETID', table_names=['TARGETPHOT', 'LSDR9'])
    
    #if len(N) > 0:
    #lsid_new = (new[N]['RELEASE'].astype(np.int64) << 40) | (new[N]['BRICKID'].astype(np.int64) << 16) | (new[N]['OBJID'].astype(np.int64))
    out_new = join(targetphot_ontile['TARGETID', 'RA', 'DEC', 'RELEASE', 'BRICKNAME', 'BRICKID', 'BRICK_OBJID'],
                   new['TARGETID', 'RA', 'DEC', 'RELEASE', 'BRICKNAME', 'BRICKID', 'OBJID', 'LS_ID'],
                   keys='TARGETID', table_names=['TARGETPHOT', 'LSDR9'])

80611 Nold=95/4198 (2.26%) Nnew=0/4198 (0.00%)
80612 Nold=62/4199 (1.48%) Nnew=0/4199 (0.00%)
80616 Nold=210/4193 (5.01%) Nnew=0/4193 (0.00%)


In [6]:
out_old[out_old['TARGETID'] == 39628509856928374]

TARGETID,RA_TARGETPHOT,DEC_TARGETPHOT,RELEASE_TARGETPHOT,BRICKNAME_TARGETPHOT,BRICKID_TARGETPHOT,BRICK_OBJID,RA_LSDR9,DEC_LSDR9,RELEASE_LSDR9,BRICKNAME_LSDR9,BRICKID_LSDR9,OBJID,LS_ID
int64,float64,float64,int16,str8,int32,int32,float64,float64,int16,str8,int32,int32,int64


In [7]:
out_new[out_new['TARGETID'] == 39628509856928374]

TARGETID,RA_TARGETPHOT,DEC_TARGETPHOT,RELEASE_TARGETPHOT,BRICKNAME_TARGETPHOT,BRICKID_TARGETPHOT,BRICK_OBJID,RA_LSDR9,DEC_LSDR9,RELEASE_LSDR9,BRICKNAME_LSDR9,BRICKID_LSDR9,OBJID,LS_ID
int64,float64,float64,int16,str8,int32,int32,float64,float64,int16,str8,int32,int32,int64
